In [1]:
import requests

def get_cities(estado):
    url = f'https://dash-saude-mongo.elsvital.dev/api/v1/cities/{estado}'
    headers = {'accept': 'application/json'}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        cities = response.json()
        return cities
    else:
        print(response.status_code)
        print(response.text)
        return None

print(get_cities('SP'))

['ADAMANTINA', 'ADOLFO', 'AGUAÍ', 'AGUDOS', 'ALAMBARI', 'ALFREDO MARCONDES', 'ALTAIR', 'ALTINÓPOLIS', 'ALTO ALEGRE', 'ALUMÍNIO', 'ALVINLÂNDIA', 'AMERICANA', 'AMPARO', 'AMÉRICO BRASILIENSE', 'AMÉRICO DE CAMPOS', 'ANALÂNDIA', 'ANDRADINA', 'ANGATUBA', 'ANHEMBI', 'ANHUMAS', 'APARECIDA', "APARECIDA D'OESTE", 'APIAÍ', 'ARAMINA', 'ARANDU', 'ARAPEÍ', 'ARARAQUARA', 'ARARAS', 'ARAÇARIGUAMA', 'ARAÇATUBA', 'ARAÇOIABA DA SERRA', 'ARCO-ÍRIS', 'AREALVA', 'AREIAS', 'AREIÓPOLIS', 'ARIRANHA', 'ARTUR NOGUEIRA', 'ARUJÁ', 'ASPÁSIA', 'ASSIS', 'ATIBAIA', 'AURIFLAMA', 'AVANHANDAVA', 'AVARÉ', 'AVAÍ', 'BADY BASSITT', 'BALBINOS', 'BANANAL', 'BARBOSA', 'BARIRI', 'BARRA BONITA', 'BARRA DO CHAPÉU', 'BARRA DO TURVO', 'BARRETOS', 'BARRINHA', 'BARUERI', 'BARÃO DE ANTONINA', 'BASTOS', 'BATATAIS', 'BAURU', 'BEBEDOURO', 'BENTO DE ABREU', 'BERNARDINO DE CAMPOS', 'BERTIOGA', 'BILAC', 'BIRIGUI', 'BIRITIBA-MIRIM', 'BOA ESPERANÇA DO SUL', 'BOCAINA', 'BOFETE', 'BOITUVA', 'BOM JESUS DOS PERDÕES', 'BOM SUCESSO DE ITARARÉ', 'BORA

In [8]:
import chardet

# Abre o arquivo em modo binário e tenta detectar a codificação
with open('data/producao.csv', 'rb') as f:
    result = chardet.detect(f.read())
    print(result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [59]:
# ler o arquivo data/producao.csv extrair uma lista de municipios, acompanhados do ibge_id e do estado

import pandas as pd

df = pd.read_csv('data/producao.csv', sep=',', encoding='utf-8',)

df.head()

,Uf,Ibge,Municipio,Asma,Desnutrição,Diabetes,DPOC,Hipertensão arterial,Obesidade,Pré-natal,...,Encaminhamento intersetorial,Encaminhamento p/ CAPS,Encaminhamento p/ internação h,Encaminhamento p/ serviço de a,Encaminhamento p/ serviço espe,Encaminhamento p/ urgência,Avaliação / diagnóstico,Procedimentos clínicos / terap,Prescrição terapêutica,Data
0,PI,220900.0,RIO GRANDE DO PIAUÍ,1.0,2.0,5.0,1.0,25.000,2.0,32.0,...,0.0,0.0,0.0,0.0,7.000,0.0,2.0,122.0,3.0,2021-05-01
1,PR,410520.0,CERRO AZUL,2.0,2.0,145.0,2.0,280.000,39.0,49.0,...,23.0,0.0,0.0,0.0,0.000,0.0,64.0,0.0,0.0,2021-05-01
2,RJ,330190.0,ITABORAÍ,86.0,15.0,970.0,28.0,2.964,171.0,586.0,...,2.0,35.0,2.0,2.0,1.192,16.0,180.0,26.0,385.0,2021-05-01
3,RS,431410.0,PASSO FUNDO,161.0,52.0,594.0,51.0,1.211,269.0,853.0,...,0.0,0.0,0.0,0.0,825.000,0.0,0.0,0.0,0.0,2021-05-01
4,GO,520630.0,CRISTIANÓPOLIS,5.0,0.0,68.0,1.0,168.000,15.0,11.0,...,0.0,0.0,0.0,0.0,9.000,0.0,0.0,0.0,0.0,2021-05-01


In [60]:
df = df[['Uf', 'Municipio', 'Ibge']]

# convert the column 'Ibge' to int
df['Ibge']  = df['Ibge'].astype(int)

# rename columns
df = df.rename(columns={'Uf': 'estado', 'Municipio': 'municipio', 'Ibge': 'ibge'})

# remove duplicates
df = df.drop_duplicates()

# salvar em json
df.to_json('data/municipios.json', orient='records')

In [19]:
import json

def get_ibge_code(estado, municipio):
    with open('data/municipios.json', 'r', encoding='utf-8') as file:
        municipios = json.load(file)
    
    for item in municipios:
        if item['estado'] == estado and item['municipio'] == municipio:
            return item['ibge']
    
    return None

# Exemplo de uso
estado = 'SP'
mun = 'CAMPINAS'
ibge_code = get_ibge_code(estado, mun)
print(f'O código IBGE para {mun}, {estado} é {ibge_code}')

O código IBGE para CAMPINAS, SP é 350950


In [ ]:
# função para retornar json a par

In [3]:
import requests

def get_atendimentos(estado, cidade):
    """Função para obter os dados de atendimentos"""
    url = "https://dash-saude-mongo.elsvital.dev/api/v1/atendimentos"
    if estado is not None:
        url = f"https://dash-saude-mongo.elsvital.dev/api/v1/atendimentos/states/{estado}"
    if cidade is not None:
        ibge_code = get_ibge_code(estado, cidade)
        url = f"https://dash-saude-mongo.elsvital.dev/api/v1/atendimentos/cities/{ibge_code}"

    print('Fazendo request para:', url)

    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(response.status_code)
        print(response.text)
        return None

# Exemplo de uso
atendimentos = get_atendimentos('SP', None)
print(atendimentos)

Fazendo request para: https://dash-saude-mongo.elsvital.dev/api/v1/atendimentos/states/SP
{'enfermeiro': {'2013': {'ABR': 4, 'AGO': 559, 'DEZ': 174, 'JUL': 325, 'JUN': 2, 'MAI': 6, 'NOV': 570, 'OUT': 519, 'SET': 415}, '2014': {'ABR': 5749, 'AGO': 59529, 'DEZ': 33321, 'FEV': 1722, 'JAN': 760, 'JUL': 53035, 'JUN': 11363, 'MAI': 8556, 'MAR': 3013, 'NOV': 44703, 'OUT': 67819, 'SET': 60520}, '2015': {'ABR': 55928, 'AGO': 127716, 'DEZ': 136563, 'FEV': 43607, 'JAN': 45483, 'JUL': 103911, 'JUN': 103168, 'MAI': 78365, 'MAR': 57578, 'NOV': 163948, 'OUT': 152465, 'SET': 150122}, '2016': {'ABR': 272728, 'AGO': 314883, 'DEZ': 194784, 'FEV': 254904, 'JAN': 208746, 'JUL': 273963, 'JUN': 304277, 'MAI': 250716, 'MAR': 343631, 'NOV': 246327, 'OUT': 280487, 'SET': 276791}, '2017': {'ABR': 499868, 'AGO': 807837, 'DEZ': 376253, 'FEV': 278307, 'JAN': 312534, 'JUL': 485956, 'JUN': 622305, 'MAI': 495139, 'MAR': 392346, 'NOV': 905712, 'OUT': 691973, 'SET': 707594}, '2018': {'ABR': 509888, 'AGO': 880924, 'DEZ':

In [70]:
import pandas as pd
# Ler o arquivo de cadastro
df = pd.read_csv('../etl/data/consolidado/Cadastro.csv', sep=',', encoding='utf-8')



In [72]:
# Filtrar pelos dados do último ano
cadastro_df = df
cadastro_ultimo_ano = cadastro_df[cadastro_df['Ano'] == cadastro_df['Ano'].max()]

# Remover coluna Mes
cadastro_ultimo_ano = cadastro_ultimo_ano.drop(columns=['Mes'])

# remover duplicados
cadastro_ultimo_ano = cadastro_ultimo_ano.drop_duplicates(subset=['IBGE'], keep='last')

# Converter para JSON
cadastro_ultimo_ano.to_json('data/cadastros.json', orient='records')

cadastro_ultimo_ano

,Uf,IBGE,Municipio,Cadastros,Ano
267074,RO,110015,OURO PRETO DO OESTE,39901,2024
267075,RO,110050,NOVO HORIZONTE DO OESTE,8608,2024
267076,PR,410720,DOIS VIZINHOS,45748,2024
267077,RS,430520,CERRO LARGO,13600,2024
267078,RS,430580,CONSTANTINA,11228,2024
...,...,...,...,...,...
272639,CE,230990,PACUJÁ,6760,2024
272640,PB,251385,SANTO ANDRÉ,3139,2024
272641,MG,310040,ACAIACA,4582,2024
272642,MG,313540,JECEABA,6350,2024


In [73]:
cadastro_ultimo_ano.drop_duplicates(subset=['IBGE'], keep='last')


,Uf,IBGE,Municipio,Cadastros,Ano
267074,RO,110015,OURO PRETO DO OESTE,39901,2024
267075,RO,110050,NOVO HORIZONTE DO OESTE,8608,2024
267076,PR,410720,DOIS VIZINHOS,45748,2024
267077,RS,430520,CERRO LARGO,13600,2024
267078,RS,430580,CONSTANTINA,11228,2024
...,...,...,...,...,...
272639,CE,230990,PACUJÁ,6760,2024
272640,PB,251385,SANTO ANDRÉ,3139,2024
272641,MG,310040,ACAIACA,4582,2024
272642,MG,313540,JECEABA,6350,2024


In [19]:
import json

def get_population(estado, municipio):
    with open('data/cadastros.json', 'r', encoding='utf-8') as file:
        cadastros = json.load(file)
    
    if municipio is None and estado is None:
        # soma de todas as populações
        total = 0
        for item in cadastros:
            total += item['Cadastros']
        return total
    
    if municipio is None:
        # soma da população do estado
        total = 0
        for item in cadastros:
            if item['Uf'] == estado:
                total += item['Cadastros']
        return total

    for item in cadastros:
        if item['Uf'] == estado and item['Municipio'] == municipio:
            return item['Cadastros']
    
    return None


populacao = get_population('SP', 'CAMPINAS')
print(f'A população é {populacao}')

A população é 931961


In [74]:
# Converter para JSON
cadastro_ultimo_ano.to_json('data/cadastros.json', orient='records')

In [15]:
import os

estado = 'AM'

# get shapefile
arquivos = os.listdir(f'../mapas/Estados/{estado}')

shapefile = [arquivo for arquivo in arquivos if arquivo.endswith('.shp')][0]
shapefile = f'../mapas/Estados/{estado}/{shapefile}'
shapefile

'../mapas/Estados/AM/AM_Municipios_2022.shp'

In [ ]:
import os

find /caminho/do/diretorio -type f -size +100M
# Listar todos os arquivos .zip no diretório
zip_files = [arquivo for arquivo in arquivos if arquivo.endswith('.zip')]

# Comando para descompactar cada arquivo .zip
for zip_file in zip_files:
    os.system(f'unzip {zip_file} -d /caminho/do/diretorio')

In [17]:
import geopandas as gpd

gpd.read_file(shapefile)

,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry
0,1300029,Alvarães,AM,5923.461,"POLYGON ((-64.71229 -3.27608, -64.7162 -3.2769..."
1,1300060,Amaturá,AM,4754.109,"POLYGON ((-67.94285 -3.32959, -67.87343 -3.332..."
2,1300086,Anamã,AM,2446.121,"POLYGON ((-61.51715 -3.6848, -61.54102 -3.6842..."
3,1300102,Anori,AM,6036.380,"POLYGON ((-61.86552 -3.97394, -61.88421 -3.965..."
4,1300144,Apuí,AM,54240.545,"POLYGON ((-60.07249 -8.80016, -60.08515 -8.800..."
...,...,...,...,...,...
57,1304203,Tefé,AM,23692.223,"POLYGON ((-64.45763 -3.42909, -64.45566 -3.434..."
58,1304237,Tonantins,AM,6446.894,"POLYGON ((-67.29106 -2.58411, -67.29104 -2.603..."
59,1304260,Uarini,AM,10274.677,"POLYGON ((-64.89493 -2.86025, -64.89588 -2.865..."
60,1304302,Urucará,AM,27901.962,"POLYGON ((-57.68172 -2.23394, -57.67938 -2.234..."


In [18]:
mapa_mun = gpd.read_file(shapefile)

mapa_mun.iloc[0]['NM_MUN']

'Alvarães'

In [25]:
mapa_mun = gpd.read_file(shapefile)
# filtrar pelo municio Alvarães
mapa_mun = mapa_mun[mapa_mun['NM_MUN'] == 'Alvarães']

mapa_mun = mapa_mun[["CD_MUN", 'NM_MUN', 'SIGLA_UF',  "geometry"]]

mapa_mun["geometry"] = mapa_mun["geometry"].simplify(tolerance=0.01)

mapa_mun['value'] = 1



fig = px.choropleth(
    mapa_mun,
        geojson=mapa_mun.geometry,  # Usar a geometria do shapefile
        locations=mapa_mun.index,  # Nome da coluna do DataFrame
        hover_name='NM_MUN',
        hover_data={"NM_MUN": False,  # Mostrar nome do município
                    "SIGLA_UF": False,  # Não mostrar a sigla do estado
                    "value": False},
        color='value',
        color_continuous_scale=["#80B0DC","#80B0DC","#80B0DC",],
    )

# Ajustar as configurações do mapa
fig.update_geos(fitbounds="locations", visible=False, )
fig.update_layout(margin={"r": 0, "t": 50, "l": 0, "b": 0}, coloraxis_showscale=False)
# Melhorar o hover ao remover o índice
fig.update_traces(hovertemplate="<b>%{hovertext}</b><extra></extra>")


fig.show()

In [38]:
import geopandas as gpd
import plotly.express as px

# shapefile_uf = "../mapas/BR_UF_2022/BR_UF_2022.shp"

#pegar o caminho do arquivo



shapefile_uf = "mapas/Estados/BR_UF_2022.shp"
brasil_estados = gpd.read_file(shapefile)


mapa_uf = brasil_estados[["SIGLA_UF", "geometry"]]

mapa_uf["geometry"] = brasil_estados["geometry"].simplify(tolerance=0.01)

# gerar um valor aleatório para cada estado
import random
mapa_uf['value'] = 1

# Converte para GeoJSON
gdf_json = mapa_uf.to_json()


fig = px.choropleth(
        mapa_uf,
        geojson=mapa_uf.geometry,  # Usar a geometria do shapefile
        locations=mapa_uf.index,  # Nome da coluna do DataFrame
        hover_name='SIGLA_UF',
        title=f"Atendimentos de por Estado no Brasil - 2023",
        hover_data={"SIGLA_UF": False, "value": False, 'index': False},
        color='value',
        color_continuous_scale=["#B36CA3","#B36CA3","#B36CA3",],
    )

# Ajustar as configurações do mapa
fig.update_geos(fitbounds="locations", visible=False, )
fig.update_layout(margin={"r": 0, "t": 50, "l": 0, "b": 0}, coloraxis_showscale=False)


fig.show()

/home/daniel/anaconda3/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



ValueError: Value of 'hover_data_2' is not the name of a column in 'data_frame'. Expected one of ['SIGLA_UF', 'geometry', 'value'] but received: index
 To use the index, pass it in directly as `df.index`.

In [23]:
# criar uma lista com os estados e os indices do dataframe
estados = mapa_uf['SIGLA_UF'].tolist()
indices = mapa_uf.index.tolist()

# criar um dicionário com os estados e os indices
estados_indices = dict(zip(indices, estados))
estados_indices

{0: 'AC',
 1: 'AM',
 2: 'PA',
 3: 'AP',
 4: 'TO',
 5: 'MA',
 6: 'PI',
 7: 'CE',
 8: 'RN',
 9: 'PB',
 10: 'PE',
 11: 'AL',
 12: 'SE',
 13: 'BA',
 14: 'MG',
 15: 'ES',
 16: 'RJ',
 17: 'SP',
 18: 'PR',
 19: 'SC',
 20: 'RS',
 21: 'MS',
 22: 'MT',
 23: 'GO',
 24: 'DF',
 25: 'RO',
 26: 'RR'}

In [4]:
# Mapeamento dos meses para seus números correspondentes
mes_map = {
    "JAN": 1,
    "FEV": 2,
    "MAR": 3,
    "ABR": 4,
    "MAI": 5,
    "JUN": 6,
    "JUL": 7,
    "AGO": 8,
    "SET": 9,
    "OUT": 10,
    "NOV": 11,
    "DEZ": 12,
}
# Mapeamento dos meses para seus trimestres correspondentes
trimestre_map = {
    "JAN": "T1",
    "FEV": "T1",
    "MAR": "T1",
    "ABR": "T2",
    "MAI": "T2",
    "JUN": "T2",
    "JUL": "T3",
    "AGO": "T3",
    "SET": "T3",
    "OUT": "T4",
    "NOV": "T4",
    "DEZ": "T4",
}

trimestre_map_num = {
    1 : "T1",
    2 : "T1",
    3 : "T1",
    4 : "T2",
    5 : "T2",
    6 : "T2",
    7 : "T3",
    8 : "T3",
    9 : "T3",
    10 : "T4",
    11 : "T4",
    12 : "T4",
}

def update_layout_chart(chart, title, type):
    # Para atualizar o layout do gráfico
    #    chart -> o gráfico que vamos alterar
    #    title -> string com o nome que deve aparecer no label do gráfico
    #    type -> string para saber em qual agregação estamos ['brasil', 'estado', 'regiao_saude', 'municipio']
    # retorna o gráfico atualizado

    color = type_color_map.get(type, [None])[1]

    chart.update_traces(
        textposition="outside",
        marker_color=color,
        hoverlabel=dict(bgcolor="#FFFFFF", font_color="#343A40", font_size=12),
        hovertemplate=f"<b>%{{y:,.0f}}</b><br>{title} em %{{x}}<extra></extra>",
    )

    chart.update_layout(
        xaxis_title=None,
        yaxis_title=None,
        plot_bgcolor="#FFFFFF",
        yaxis=dict(showticklabels=False),
        margin=dict(l=35, r=35, t=60, b=40),
    )

    return chart

def get_df_atendimentos(json_data, populacao=None):
    """Função para transformar um json de atendimento em um df que será utilizado para gerar os gráficos"""
    # Para transformar um json de atendimento em um df que será utilizado para gerar os gráficos
    #    json_data -> json que contem os dados de atendimento
    # retorna o df
    dados = []

    # Iterar sobre os diferentes tipos (enfermeiro, médico, etc.)
    for profissional, anos in json_data.items():
        # Iterar sobre os anos (2013, 2014, etc.)
        for ano, meses in anos.items():
            # Iterar sobre os meses e seus valores
            for mes, valor in meses.items():
                # Adicionar uma linha para cada entrada
                dados.append(
                    [
                        profissional,
                        int(ano),
                        trimestre_map[mes],
                        mes_map[mes],
                        valor,
                    ]
                )

    # Criar o DataFrame com as colunas: profissional, ano, mês, valor
    df = pd.DataFrame(
        dados, columns=["profissional", "ano", "trimestre", "mes", "valor"]
    )
    # df['trimestre'] = df['mes'].apply(calcular_trimestre)
    df["ano_mes"] = df["mes"].astype(str) + "/" + df["ano"].astype(str)
    df["ano_trimestre"] = (
        df["trimestre"].astype(str) + "/" + df["ano"].astype(str)
    )

    # normalizar valores pelo total da população (1000 habitantes)
    if populacao is not None:
        populacao = populacao / 1000
        df["valor"] = df["valor"] / populacao
        df["valor"] = df["valor"].astype(int)

    return df

def get_atendimentos(estado, cidade):
    """Função para obter os dados de atendimentos"""
    url = "https://dash-saude-mongo.elsvital.dev/api/v1/atendimentos"
    if estado is not None:
        url = f"https://dash-saude-mongo.elsvital.dev/api/v1/atendimentos/states/{estado}"
    if cidade is not None:
        ibge_code = get_ibge_code(estado, cidade)
        url = f"https://dash-saude-mongo.elsvital.dev/api/v1/atendimentos/cities/{ibge_code}"

    print("Fazendo request para:", url)

    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(response.status_code)
        print(response.text)
        return None

type_color_map = {
    "brasil": ["#B36CA3", "#632956", "#3B032F"],
    "estado": ["#80B0DC", "#34679A", "#11173F"],
    "regiao_saude": ["#97C471", "#2B7B6F", "#11302B"],
    "municipio": ["#FFC20D", "#F7941C", "#A7620E"],
}

In [9]:
import requests
import pandas as pd
import plotly.express as px

def get_chart_by_quarter(df, title, type):
    # Retorna o gráfico de barras com o total acumulado dos últimos 6 trimestres de dados
    #    df -> dados para gerar o gráfico que deve conter ['ano_trimestre', 'ano', 'trimestre', 'valor']
    #    title -> string com o nome que deve aparecer no label do gráfico
    #    type -> string para saber em qual agregação estamos ['brasil', 'estado', 'regiao_saude', 'municipio']
    # retorna o gráfico gerado

    # Agrupar os dados por ano e quarter somando os valores
    df_grouped = (
        df.groupby(["ano_trimestre", "ano", "trimestre"], observed=True)[
            "valor"
        ]
        .sum()
        .reset_index()
    )
    df_grouped["ano_order"] = df_grouped["ano"].astype(str) + df_grouped[
        "trimestre"
    ].astype(str).str.replace("T", "")
    df_grouped = df_grouped.sort_values("ano_order")
    df_filtered = df_grouped.tail(12)

    # Criar gráfico de barras
    chart = px.bar(
        df_filtered,
        x="ano_trimestre",
        y="valor",
        text_auto=".2s",
        title=f"{title.capitalize()} por Trimestre",
    )

    # Atualizar para o layout padrão
    chart = update_layout_chart(chart, title, type)


    return chart

atendimentos = get_atendimentos(None, None)
df = get_df_atendimentos(atendimentos)
chart = get_chart_by_quarter(df, 'Atendimentos', 'brasil')

Fazendo request para: https://dash-saude-mongo.elsvital.dev/api/v1/atendimentos


In [103]:
title =''
type = 'brasil'
df_grouped = (
    df.groupby(["ano_trimestre", "ano", "trimestre"], observed=True)[
        "valor"
    ]
    .sum()
    .reset_index()
)
df_grouped["ano_order"] = df_grouped["ano"].astype(str) + df_grouped[
    "trimestre"
].astype(str).str.replace("T", "")
df_grouped = df_grouped.sort_values("ano_order")
df_filtered = df_grouped.tail(12)

# Criar gráfico de barras
chart = px.bar(
    df_filtered,
    x="ano_trimestre",
    y="valor",
    text_auto=".2s",
    title=f"{title.capitalize()} por Trimestre",
)

# Atualizar para o layout padrão
chart = update_layout_chart(chart, title, type)

chart.show()


In [110]:
df.groupby(["ano_trimestre", "ano", "trimestre", "mes"], observed=True)[
        "valor"
    ].sum().reset_index().sort_values(["ano", "mes"])

,ano_trimestre,ano,trimestre,mes,valor
33,T2/2013,2013,T2,4,1244
34,T2/2013,2013,T2,5,591
35,T2/2013,2013,T2,6,2324
69,T3/2013,2013,T3,7,12591
70,T3/2013,2013,T3,8,19388
...,...,...,...,...,...
32,T1/2024,2024,T1,3,30701422
66,T2/2024,2024,T2,4,37395734
67,T2/2024,2024,T2,5,29362662
68,T2/2024,2024,T2,6,28517242


In [142]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.graph_objects as go

# Função para formatar números grandes
def formatar_numero(numero):
    if numero >= 1_000_000_000:
        return f"{numero / 1_000_000_000:.1f}B"
    elif numero >= 1_000_000:
        return f"{numero / 1_000_000:.1f}M"
    elif numero >= 1_000:
        return f"{numero / 1_000:.1f}K"
    return str(numero)

def get_chart_by_quarter(df, title, type):
    # Retorna o gráfico de barras com o total acumulado dos últimos 6 trimestres de dados
    #    df -> dados para gerar o gráfico que deve conter ['ano_trimestre', 'ano', 'trimestre', 'valor']
    #    title -> string com o nome que deve aparecer no label do gráfico
    #    type -> string para saber em qual agregação estamos ['brasil', 'estado', 'regiao_saude', 'municipio']
    # retorna o gráfico gerado

    # Agrupar os dados por ano e quarter somando os valores
    df_grouped = (
        df.groupby(["ano_trimestre", "ano", "trimestre"], observed=True)[
            "valor"
        ]
        .sum()
        .reset_index()
    )
    df_grouped["ano_order"] = df_grouped["ano"].astype(str) + df_grouped[
        "trimestre"
    ].astype(str).str.replace("T", "")
    df_grouped = df_grouped.sort_values("ano_order")
    df_filtered = df_grouped.tail(12)

    # Criar gráfico de barras
    chart = px.bar(
        df_filtered,
        x="ano_trimestre",
        y="valor",
        text_auto=".2s",
        title=f"{title.capitalize()} por Trimestre",
    )

    # Previsão SARIMA
    df_grouped_mes = df.groupby(["ano_trimestre", "ano", "trimestre", "mes"], observed=True)[
        "valor"
    ].sum().reset_index().sort_values(["ano", "mes"])
    df_sarima = df_grouped_mes[:-1]  # Desconsiderar o último mês
    model = SARIMAX(df_sarima['valor'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    results = model.fit()
    forecast = results.get_forecast(steps=6)

    # Calcular os valores de ano e mes para as proximas previsoes
    last_year = df_sarima['ano'].max()
    last_month = df_sarima[df_sarima['ano'] == last_year]['mes'].max()

    forecast_month = []
    forecast_year = []
    for i in range(1, 7):
        if last_month == 12:
            last_month = 1
            last_year += 1
        else:
            last_month += 1
        forecast_month.append(last_month)
        forecast_year.append(last_year)

    forecast_trimestre = [trimestre_map_num[m] for m in forecast_month]

    
    forecast_index = [f"{t}/{y}" for t, y in zip(forecast_trimestre, forecast_year)]
    forecast_values = forecast.predicted_mean

    # Adicionar previsões ao gráfico
    forecast_df = pd.DataFrame({'ano_trimestre': forecast_index, 'valor': forecast_values})

    forecast_df = forecast_df.groupby("ano_trimestre")["valor"].sum().reset_index()

    forecast_df['valor'] = forecast_df['valor'].astype(str)
    
    # Adiciona os valores dos pontos
    chart.add_trace(go.Scatter(
        x=forecast_df['ano_trimestre'], 
        y=forecast_df['valor'], 
        mode='lines+markers+text',  # Adiciona linha, marcadores e texto
        text=[f'{y:.2s}' for y in forecast_df['valor']],              # O valor do ponto como texto
        textposition='top center',   # Posição do texto
        name='Previsão',      # Nome da série
    ))
    # Atualizar para o layout padrão
    # chart = update_layout_chart(chart, title, type)

    return chart

# atendimentos = get_atendimentos(None, None)
# df = get_df_atendimentos(atendimentos)
chart = get_chart_by_quarter(df, 'Atendimentos', 'brasil')
chart.show()

/tmp/ipykernel_663/3414722418.py:47: FutureWarning:

In a future version of pandas all arguments of Series.sort_values will be keyword-only



TypeError: unhashable type: 'list'

In [14]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.graph_objects as go
import sys
import os
import contextlib


# Função para suprimir a saída
@contextlib.contextmanager
def suppress_stdout_stderr():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr


def preprocess_data(df):
    df_grouped = (
        df.groupby(["ano_trimestre", "ano", "trimestre"], observed=True)[
            "valor"
        ]
        .sum()
        .reset_index()
    )
    df_grouped["ano_order"] = df_grouped["ano"].astype(str) + df_grouped[
        "trimestre"
    ].astype(str).str.replace("T", "")
    df_grouped = df_grouped.sort_values("ano_order")
    return df_grouped.tail(12)

def create_bar_chart(df_filtered, title, type):
    chart = px.bar(
        df_filtered,
        x="ano_trimestre",
        y="valor",
        text_auto=".2s",
        title=f"{title.capitalize()} por Trimestre",
    )
    chart = update_layout_chart(chart, title, type)
    return chart

def preprocess_sarima_data(df):
    df_grouped_mes = df.groupby(["ano_trimestre", "ano", "trimestre", "mes"], observed=True)[
        "valor"
    ].sum().reset_index().sort_values(["ano", "mes"])
    df_grouped_mes.reset_index(drop=True, inplace=True)
    return df_grouped_mes[:-1]  # Desconsiderar o último mês

def fit_sarima_model(df_sarima):
    with suppress_stdout_stderr():
        model = SARIMAX(df_sarima['valor'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
        results = model.fit()
    return results.get_forecast(steps=6)

def generate_forecast_dates(df_sarima):
    last_year = df_sarima['ano'].max()
    last_month = df_sarima[df_sarima['ano'] == last_year]['mes'].max()

    forecast_month = []
    forecast_year = []
    for i in range(1, 7):
        if last_month == 12:
            last_month = 1
            last_year += 1
        else:
            last_month += 1
        forecast_month.append(last_month)
        forecast_year.append(last_year)

    forecast_trimestre = [trimestre_map_num[m] for m in forecast_month]
    forecast_index = [f"{t}/{y}" for t, y in zip(forecast_trimestre, forecast_year)]
    return forecast_index

def create_forecast_df(forecast_index, forecast_values):
    forecast_df = pd.DataFrame({'ano_trimestre': forecast_index, 'valor': forecast_values})
    forecast_df = forecast_df.groupby("ano_trimestre")["valor"].sum().reset_index()
    forecast_df['valor'] = forecast_df['valor'].astype(str)
    return forecast_df

def forecast_sarima(df):
    df_sarima = preprocess_sarima_data(df)
    forecast = fit_sarima_model(df_sarima)
    forecast_index = generate_forecast_dates(df_sarima)
    forecast_values = forecast.predicted_mean
    return create_forecast_df(forecast_index, forecast_values)

def add_forecast_to_chart(chart, forecast_df, type):
    chart.add_trace(go.Scatter(
        x=forecast_df['ano_trimestre'], 
        y=forecast_df['valor'], 
        mode='lines+markers+text',  
        text=[f'{y:.2s}' for y in forecast_df['valor']],              
        textposition='top center',   
        name='Previsão',
        line = dict(color=type_color_map.get(type, [None])[1], width=2, dash='dash'),
        hovertemplate=f"<b>%{{y:,.0f}}</b><br>Previsão para o %{{x}}<extra></extra>",
        hoverlabel=dict(bgcolor="#FFFFFF", font_color="#343A40", font_size=12),     
    ))
    return chart

def get_chart_by_quarter(df, title, type):
    df_filtered = preprocess_data(df)
    chart = create_bar_chart(df_filtered, title, type)
    forecast_df = forecast_sarima(df)
    chart = add_forecast_to_chart(chart, forecast_df, type)
    return chart

chart = get_chart_by_quarter(df, 'Atendimentos', 'brasil')
chart.show()

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.45761D+01    |proj g|=  3.36428D-02

At iterate    5    f=  1.45566D+01    |proj g|=  9.17510D-02

At iterate   10    f=  1.45332D+01    |proj g|=  2.52341D-03

At iterate   15    f=  1.45331D+01    |proj g|=  1.76163D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     15     17      1     0     0   1.762D-05   1.453D+01
  F =   14.533053094136291     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [ ]:
def preprocess_sarima_data(df):
    df_grouped_mes = df.groupby(["ano_trimestre", "ano", "trimestre", "mes"], observed=True)[
        "valor"
    ].sum().reset_index().sort_values(["ano", "mes"])
    df_grouped_mes.reset_index(drop=True, inplace=True)
    return df_grouped_mes[:-1]  # Desconsiderar o último mês

def fit_sarima_model(df_sarima):
    model = SARIMAX(df_sarima['valor'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    results = model.fit()
    return results.get_forecast(steps=6)

def generate_forecast_dates(df_sarima):
    last_year = df_sarima['ano'].max()
    last_month = df_sarima[df_sarima['ano'] == last_year]['mes'].max()

    forecast_month = []
    forecast_year = []
    for i in range(1, 7):
        if last_month == 12:
            last_month = 1
            last_year += 1
        else:
            last_month += 1
        forecast_month.append(last_month)
        forecast_year.append(last_year)

    forecast_trimestre = [trimestre_map_num[m] for m in forecast_month]
    forecast_index = [f"{t}/{y}" for t, y in zip(forecast_trimestre, forecast_year)]
    return forecast_index

def create_forecast_df(forecast_index, forecast_values):
    forecast_df = pd.DataFrame({'ano_trimestre': forecast_index, 'valor': forecast_values})
    forecast_df = forecast_df.groupby("ano_trimestre")["valor"].sum().reset_index()
    forecast_df['valor'] = forecast_df['valor'].astype(str)
    return forecast_df

def forecast_sarima(df):
    df_sarima = preprocess_sarima_data(df)
    forecast = fit_sarima_model(df_sarima)
    forecast_index = generate_forecast_dates(df_sarima)
    forecast_values = forecast.predicted_mean
    return create_forecast_df(forecast_index, forecast_values)

In [148]:
type_color_map.get(type, [None])[1]

'#632956'

In [144]:
df_sarima = df.groupby(["ano_trimestre", "ano", "trimestre", "mes"], observed=True)["valor"].sum().reset_index().sort_values(["ano", "mes"])
df_sarima.reset_index(drop=True, inplace=True)
df_sarima

,ano_trimestre,ano,trimestre,mes,valor
0,T2/2013,2013,T2,4,1244
1,T2/2013,2013,T2,5,591
2,T2/2013,2013,T2,6,2324
3,T3/2013,2013,T3,7,12591
4,T3/2013,2013,T3,8,19388
...,...,...,...,...,...
131,T1/2024,2024,T1,3,30701422
132,T2/2024,2024,T2,4,37395734
133,T2/2024,2024,T2,5,29362662
134,T2/2024,2024,T2,6,28517242


In [8]:
import pandas as pd

# Import the Excel file
df_reg_mun = pd.read_excel('data/reg_mun.xlsx')

# Display the first few rows of the dataframe
df_reg_mun.head()

,uf,cidade,ibge,regiao,no_regiao
0,AC,Acrelândia,120001,12002,Baixo Acre e Purus
1,AC,Assis Brasil,120005,12001,Alto Acre
2,AC,Brasiléia,120010,12001,Alto Acre
3,AC,Bujari,120013,12002,Baixo Acre e Purus
4,AC,Capixaba,120017,12002,Baixo Acre e Purus


In [7]:
cadastros_df = pd.read_json('data/cadastros.json', orient='records')
cadastros_df.head()

,Uf,IBGE,Municipio,Cadastros,Ano
0,RO,110015,OURO PRETO DO OESTE,39901,2024
1,RO,110050,NOVO HORIZONTE DO OESTE,8608,2024
2,PR,410720,DOIS VIZINHOS,45748,2024
3,RS,430520,CERRO LARGO,13600,2024
4,RS,430580,CONSTANTINA,11228,2024


In [10]:
# Ordenar cadastros_df pelo campo 'IBGE'
cadastros_df = cadastros_df.sort_values(by='IBGE').reset_index(drop=True)

# Ordenar df_reg_mun pelo campo 'ibge'
df_reg_mun = df_reg_mun.sort_values(by='ibge').reset_index(drop=True)

print(cadastros_df.head())
print(df_reg_mun.head())

   Uf    IBGE              Municipio  Cadastros   Ano
0  RO  110001  ALTA FLORESTA D'OESTE      23902  2024
1  RO  110002              ARIQUEMES      95630  2024
2  RO  110003                 CABIXI       6284  2024
3  RO  110004                 CACOAL      84637  2024
4  RO  110005             CEREJEIRAS      19663  2024
   uf                 cidade    ibge  regiao        no_regiao
0  RO  Alta Floresta D'oeste  110001   11005     Zona da Mata
1  RO              Ariquemes  110002   11001   Vale do Jamari
2  RO                 Cabixi  110003   11006         Cone Sul
3  RO                 Cacoal  110004   11002             Café
4  RO             Cerejeiras  110005   11006         Cone Sul


In [11]:
df_reg_mun['cadastros'] = cadastros_df['Cadastros']
df_reg_mun['cidade'] = df_reg_mun['cidade'].str.upper()
df_reg_mun['no_regiao'] = df_reg_mun['no_regiao'].str.upper()
# rename column cidade to municipio
df_reg_mun = df_reg_mun.rename(columns={'cidade': 'municipio'})
df_reg_mun.head()

,uf,municipio,ibge,regiao,no_regiao,cadastros
0,RO,ALTA FLORESTA D'OESTE,110001,11005,ZONA DA MATA,23902
1,RO,ARIQUEMES,110002,11001,VALE DO JAMARI,95630
2,RO,CABIXI,110003,11006,CONE SUL,6284
3,RO,CACOAL,110004,11002,CAFÉ,84637
4,RO,CEREJEIRAS,110005,11006,CONE SUL,19663


In [12]:
# salvar em json
df_reg_mun.to_json('data/reg_mun.json', orient='records')

In [2]:
import pandas as pd

# ler o arquivo data/reg_mun e dar um head
df_reg_mun = pd.read_json('data/reg_mun.json', orient='records')
df_reg_mun.head()

,uf,municipio,ibge,regiao,no_regiao,cadastros
0,RO,ALTA FLORESTA D'OESTE,110001,11005,ZONA DA MATA,23902
1,RO,ARIQUEMES,110002,11001,VALE DO JAMARI,95630
2,RO,CABIXI,110003,11006,CONE SUL,6284
3,RO,CACOAL,110004,11002,CAFÉ,84637
4,RO,CEREJEIRAS,110005,11006,CONE SUL,19663


In [15]:
# Alterar o nome do municipio de distrito federal para brasilia
df_reg_mun.loc[df_reg_mun['municipio'] == 'DISTRITO FEDERAL', 'municipio'] = 'BRASÍLIA'
print(df_reg_mun[df_reg_mun['municipio'] == 'BRASÍLIA'])

      uf municipio    ibge  regiao          no_regiao  cadastros
5569  DF  BRASÍLIA  530010   53001   DISTRITO FEDERAL    2209181


In [16]:
# Atualizar o json
df_reg_mun.to_json('data/reg_mun.json', orient='records')

In [21]:
import requests

def get_population(estado, mun):
    """Função para obter a população de um município, estado ou do Brasil"""
    with open("data/cadastros.json", "r", encoding="utf-8") as file:
        cadastros = json.load(file)

    if mun is None and estado is None:
        # soma de todas as populações
        total = 0
        for item in cadastros:
            total += item["Cadastros"]
        return total

    if mun is None:
        # soma da população do estado
        total = 0
        for item in cadastros:
            if item["Uf"] == estado:
                total += item["Cadastros"]
        return total

    for item in cadastros:
        if item["Uf"] == estado and item["Municipio"] == mun:
            return item["Cadastros"]

    return None

[2024, 2023, 2022, 2021, 2020, 2019]

In [24]:
def get_population(estado, mun):
    """Função para obter a população de um município, estado ou do Brasil"""
    if mun is None and estado is None:
        # soma de todas as populações
        total = df_reg_mun['cadastros'].sum()
        return total

    if mun is None:
        # soma da população do estado
        total = df_reg_mun[df_reg_mun['uf'] == estado]['cadastros'].sum()
        return total

    # população do município específico
    total = df_reg_mun[(df_reg_mun['uf'] == estado) & (df_reg_mun['municipio'] == mun)]['cadastros'].sum()
    return total

# Exemplo de uso
populacao = get_population(None, None)
print(f'A população é {populacao}')

A população é 181277662


In [36]:
import pandas as pd
file_path = 'data/reg_mun.json'
municipios = pd.read_json(file_path)

def get_cidades_regiao(regiao):
    """Função para obter um dicionario dos ibge e municipios de uma região"""
    # TODO implementar via API
    municipios_regiao = municipios[municipios['no_regiao'] == regiao][['ibge', 'municipio']].drop_duplicates()
    municipios_dict = dict(zip(municipios_regiao['ibge'], municipios_regiao['municipio']))
    return municipios_dict

# Exemplo de uso
regiao = 'ADAMANTINA'
municipios_regiao = get_cidades_regiao(regiao)
municipios_regiao

{350010: 'ADAMANTINA',
 351600: 'FLÓRIDA PAULISTA',
 352080: 'INÚBIA PAULISTA',
 352740: 'LUCÉLIA',
 352890: 'MARIÁPOLIS',
 353460: 'OSVALDO CRUZ',
 353490: 'PACAEMBU',
 354085: 'PRACINHA',
 354470: 'SAGRES',
 354510: 'SALMOURÃO'}

In [39]:
def get_code_regiao(estado, regiao):
    """Função para obter o código da região de um estado"""
    # verificar se a regiao está no dataframe
    cod = municipios[
        (municipios["uf"] == estado) & (municipios["no_regiao"] == regiao)
    ]["regiao"]
    if len(cod) == 0:
        print(f"Erro ao obter o código da região {regiao} no estado {estado}")
        return None
    return cod.values[0]

estado = 'MT'
regiao ='NORTE ARAGUAIA KARAJÁ'

get_code_regiao(estado, regiao)

Erro ao obter o código da região NORTE ARAGUAIA KARAJÁ no estado MT


In [40]:
municipios[
        (municipios["uf"] == estado) & (municipios["no_regiao"] == regiao)
    ]["regiao"]

5186    51009
5245    51009
5268    51009
5303    51009
5305    51009
Name: regiao, dtype: int64

In [4]:
import pandas as pd
def get_json_municipios():
    """Função para obter o json com os municípios do Brasil"""
    municipios = pd.read_json('data/reg_mun.json')
    return municipios

municipios = get_json_municipios()
municipios['cadastros']

0         23902
1         95630
2          6284
3         84637
4         19663
         ...   
5565      16632
5566      10287
5567       5145
5568       6680
5569    2209181
Name: cadastros, Length: 5570, dtype: int64

In [ ]:
# read from ../etl/data/consolidado/producao_problema.csv
df = pd.read_csv('../etl/data/consolidado/producao_problema.csv', sep=',', encoding='utf-8')
df.head()

: 